In [ ]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:10])

### 문제 2-1 : 콤마 구분 리스트 파서 활용


In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import os

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template=(
        "다음 사용자의 관심 분야와 관련된 한국의 유명한 장소나 활동 5가지를 추천하라.\n"
        "- 분야: {subject}\n"
        "- 조건: 한국 관련, 서로 다른 5개, 간결한 한국어 표기, 한 줄, 콤마로만 구분, 번호/불릿/따옴표/설명 금지\n"
        "{format_instructions}"
    ),
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

chain = prompt | llm | output_parser

def main():
    subject = input("관심 분야를 입력하세요 (예: 음식, 스포츠, 영화): ").strip()
    if not subject:
        print("빈 입력입니다. 프로그램을 종료합니다.")
        return

    raw_list = chain.invoke({"subject": subject})

    items = [s.strip() for s in raw_list if s and s.strip()]
    if len(items) > 5:
        items = items[:5]

    print(items)

if __name__ == "__main__":
    main()


### 문제 2-2 : 영화 리뷰 감정 분석기

In [ ]:
from enum import Enum
from typing import Union

from langchain.output_parsers import EnumOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

try:
    from langchain.output_parsers import OutputFixingParser
    HAS_FIXER = True
except Exception:
    HAS_FIXER = False

class Sentiment(Enum):
    긍정 = "긍정"
    부정 = "부정"
    보통 = "보통"

enum_parser = EnumOutputParser(enum=Sentiment)
format_instructions = enum_parser.get_format_instructions()

prompt = PromptTemplate(
    template=(
        "다음 영화 리뷰의 감정을 {labels} 중 하나로만 분류하라.\n"
        "- 출력은 한 단어(라벨)만, 설명/부연/구두점 금지\n"
        "- 리뷰: {review}\n"
        "{format_instructions}"
    ),
    input_variables=["review"],
    partial_variables={
        "labels": ", ".join([e.value for e in Sentiment]),
        "format_instructions": format_instructions,
    },
)

llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

if HAS_FIXER:
    fix_parser = OutputFixingParser.from_llm(parser=enum_parser, llm=llm)
    chain = prompt | llm | fix_parser
else:
    chain = prompt | llm | enum_parser

def classify_review(review: str) -> str:
    """Return one of {'긍정','부정','보통'}; fallback to strict parsing if needed."""
    try:
        result: Union[Sentiment, str] = chain.invoke({"review": review})
        # If OutputFixingParser returns raw string, normalize it.
        if isinstance(result, Sentiment):
            return result.value
        text = str(result).strip().strip('"').strip("'")
        # Strict normalize to known labels
        if text in {e.value for e in Sentiment}:
            return text
        # Last-resort strict parse
        parsed = enum_parser.parse(text)
        return parsed.value if isinstance(parsed, Sentiment) else str(parsed)
    except Exception:
        return "분석실패"

def pretty_print(results):
    """Neat table-like print."""
    for i, (review, senti) in enumerate(results, 1):
        print(f"{i}. [{senti}] {review}")

def main():
    # --- Built-in test set (요구: 여러 개의 테스트 리뷰로 검증) ---
    test_reviews = [
        "이 영화 정말 재미없어요. 시간 낭비였습니다.",
        "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
        "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.",
    ]
    results = [(r, classify_review(r)) for r in test_reviews]
    print("=== 테스트 리뷰 결과 ===")
    pretty_print(results)

    # --- Interactive mode ---
    print("\n직접 리뷰를 입력해 보세요. (빈 줄 입력 시 종료)")
    while True:
        user = input("> ").strip()
        if not user:
            break
        print(f"[{classify_review(user)}]")

if __name__ == "__main__":
    main()


### 문제 2-3 : 학생 정보 구조화 시스템


In [ ]:
from typing import List, Optional
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

try:
    from langchain.output_parsers import OutputFixingParser
    HAS_FIXER = True
except Exception:
    HAS_FIXER = False

class StudentInfo(BaseModel):
    """Structured student info extracted from a free-form self-introduction."""
    name: Optional[str] = Field(
        None, description="학생의 이름. 예: 김민수"
    )
    age: Optional[int] = Field(
        None, description="나이(정수). 예: 22"
    )
    major: Optional[str] = Field(
        None, description="전공명. 예: 컴퓨터공학"
    )
    hobbies: List[str] = Field(
        default_factory=list,
        description="취미 리스트(간결한 명사구). 예: ['게임하기','영화보기','코딩']"
    )
    goal: Optional[str] = Field(
        None, description="장래 목표 또는 포부 한 문장. 예: 훌륭한 개발자가 되는 것"
    )

parser = PydanticOutputParser(pydantic_object=StudentInfo)
format_instructions = parser.get_format_instructions()

prompt = PromptTemplate(
    template=(
        "다음 자기소개 텍스트에서 학생 정보를 추출해 구조화하라.\n"
        "필드는 name, age, major, hobbies, goal 이다.\n"
        "규칙:\n"
        "- {format_instructions}\n"
        "- age는 숫자만 반환(예: '22살' -> 22)\n"
        "- hobbies는 중복 제거, 최대 5개, 간결한 명사구로만 구성\n"
        "- 값이 불명확하면 해당 필드는 비워도 됨(null 또는 빈 리스트)\n"
        "- 한국어 표기를 유지하되 과도한 존칭/불필요 표현은 제거\n\n"
        "자기소개:\n{bio}"
    ),
    input_variables=["bio"],
    partial_variables={"format_instructions": format_instructions},
)

llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

if HAS_FIXER:
    fix_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)
    chain = prompt | llm | fix_parser
else:
    chain = prompt | llm | parser


def extract_student_info(text: str) -> StudentInfo:
    """Run the chain and return a StudentInfo Pydantic object."""
    try:
        result: StudentInfo = chain.invoke({"bio": text})

        seen, clean_h = set(), []
        for h in result.hobbies:
            item = (h or "").strip(" ,·.-").strip()
            if item and item not in seen:
                seen.add(item)
                clean_h.append(item)
        result.hobbies = clean_h[:5]
        return result
    except Exception as e:

        return StudentInfo()


def pretty_print(info: StudentInfo):
    """Neat JSON-like print (dict)."""
    print(info.model_dump())


def main():

    tests = [
       
        "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. "
        "취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다.",

        "저는 홍길동입니다. 나이는 스무둘이고 전공은 소프트웨어학부예요. "
        "여가 시간엔 등산과 독서를 합니다. 꿈은 사용자에게 사랑받는 서비스를 만드는 것입니다.",

        "이름은 박지현. 취미는 베이킹, 사진찍기. 목표는 아직 고민 중이에요.",
    ]
    print("=== 테스트 결과 ===")
    for i, t in enumerate(tests, 1):
        info = extract_student_info(t)
        print(f"\n[{i}]")
        pretty_print(info)

    print("\n직접 자기소개를 입력해 보세요. (빈 줄 입력 시 종료)")
    while True:
        bio = input("> ").strip()
        if not bio:
            break
        info = extract_student_info(bio)
        pretty_print(info)


if __name__ == "__main__":
    main()


### 문제 2-4 : 여행 계획 분석기


In [13]:
from typing import List, Dict, Any
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

try:
    from langchain.output_parsers import OutputFixingParser
    HAS_FIXER = True
except Exception:
    HAS_FIXER = False

response_schemas = [
    ResponseSchema(
        name="destination",
        description="여행지(도시/지역/국가). 예: '부산', '제주', '강릉'"
    ),
    ResponseSchema(
        name="duration",
        description="여행 기간 표현(자연어 그대로). 예: '2박 3일', '당일', '일주일'"
    ),
    ResponseSchema(
        name="budget",
        description="총 예산(자연어 그대로). 예: '30만원', '약 50만 원', '100달러 미만'"
    ),
    ResponseSchema(
        name="rating",
        description="추천도 1~5 정수 중 하나. 예: 4"
    ),
    ResponseSchema(
        name="activities",
        description="주요 활동 리스트(간결한 한국어 명사구). 예: ['해운대 바다구경','자갈치시장 회 먹기']"
    ),
]

parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

prompt = PromptTemplate(
    template=(
        "다음 여행 후기/계획 텍스트에서 핵심 정보를 추출해 구조화하라.\n"
        "필드는 destination, duration, budget, rating, activities 다섯 가지이다.\n"
        "규칙:\n"
        "- {format_instructions}\n"
        "- rating은 1~5 중 하나의 정수만 반환\n"
        "- activities는 중복 제거하고 1~6개 사이로, 간결한 명사구만 반환(설명/이모지 금지)\n"
        "- 값이 불명확하면 해당 필드는 비워도 된다(빈 문자열이나 빈 리스트 허용)\n\n"
        "텍스트:\n{review}"
    ),
    input_variables=["review"],
    partial_variables={"format_instructions": format_instructions},
)

llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

if HAS_FIXER:
    fix_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)
    chain = prompt | llm | fix_parser
else:
    chain = prompt | llm | parser

def _normalize(result: Dict[str, Any]) -> Dict[str, Any]:
    try:
        r = int(str(result.get("rating", "")).strip())
        if 1 <= r <= 5:
            result["rating"] = r
        else:
            result["rating"] = None
    except Exception:
        result["rating"] = None

    acts = result.get("activities", [])
    if not isinstance(acts, list):
        acts = [str(acts)]
    seen, clean = set(), []
    for a in acts:
        s = str(a or "").strip(" ,·.-").strip()
        if s and s not in seen:
            seen.add(s)
            clean.append(s)
    result["activities"] = clean[:6]

    for k in ["destination", "duration", "budget"]:
        v = result.get(k, "")
        result[k] = str(v).strip() if v is not None else ""

    return result


def analyze_trip(text: str) -> Dict[str, Any]:
    raw = chain.invoke({"review": text})
    return _normalize(dict(raw))


def pretty_print(d: Dict[str, Any]):
    print(d)


def main():
    sample = (
        "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 "
        "해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. "
        "정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요."
    )
    print("=== 샘플 테스트 ===")
    pretty_print(analyze_trip(sample))

    tests = [
        "제주 당일치기 다녀왔고 예산은 10만원 내외. 우도 배 타고, 협재해수욕장 산책. 별점은 5점!",
        "강릉 1박2일. 커피거리 카페 투어와 경포해변 일출 감상. 예산은 잘 기억 안 남. 추천도는 3.",
        "도쿄 4일. 스시/디즈니/팀랩. 12만엔 정도. 별점? 최고였음."
    ]
    print("\n=== 추가 테스트 ===")
    for i, t in enumerate(tests, 1):
        print(f"\n[{i}]")
        pretty_print(analyze_trip(t))

    print("\n여행 후기/계획을 입력해줘. (빈 줄이면 종료)")
    while True:
        text = input("> ").strip()
        if not text:
            break
        pretty_print(analyze_trip(text))


if __name__ == "__main__":
    main()


=== 샘플 테스트 ===
{'destination': '부산', 'duration': '2박 3일', 'budget': '30만원', 'rating': 4, 'activities': ['해운대 바다구경', '자갈치시장 회 먹기', '감천문화마을 구경']}

=== 추가 테스트 ===

[1]
{'destination': '제주', 'duration': '당일치기', 'budget': '10만원 내외', 'rating': 5, 'activities': ['우도 배 타기', '협재해수욕장 산책']}

[2]
{'destination': '강릉', 'duration': '1박2일', 'budget': '', 'rating': 3, 'activities': ['커피거리 카페 투어', '경포해변 일출 감상']}

[3]
{'destination': '도쿄', 'duration': '4일', 'budget': '12만엔 정도', 'rating': 5, 'activities': ['스시먹기', '디즈니랜드', '팀랩계획']}

여행 후기/계획을 입력해줘. (빈 줄이면 종료)
